## Main Vọc

1. With Cursor
```
@project-flashrag 
Kiểm tra xem project rag này: 
1. Pipeline như nào 
2. Các model đang được sử dụng chính? 
```

Dựa trên thông tin từ các file được cung cấp, tôi có thể phân tích pipeline và các model chính được sử dụng trong project RAG này như sau:

1. Pipeline:

Pipeline chính của project này bao gồm các bước sau:

a) Retriever: Sử dụng một mô hình retriever để tìm kiếm các đoạn văn bản liên quan từ corpus.

b) (Tùy chọn) Reranker: Nếu được bật, sẽ sử dụng một mô hình reranker để sắp xếp lại các kết quả từ retriever.

c) Generator: Sử dụng một mô hình ngôn ngữ lớn (LLM) để tạo câu trả lời dựa trên câu hỏi và các đoạn văn bản được truy xuất.

Quy trình này được thể hiện trong file `main.py`:


```183:193:project-flashrag/main.py
    golden_answers = dataset.golden_answers
    inference_rag = InferenceRAG(config)
    pred_answer_list = inference_rag.process(dataset)
    questions = dataset.question
    retrieval_results = dataset.retrieval_result
    data_writer = []
    for idx, question in enumerate(questions):
        reference = ""
        for i, element in enumerate(retrieval_results[idx]):
            reference += f"Doc({i}):\n" + element.get("contents") + "\n"
        data_writer.append([question, golden_answers[idx][0], pred_answer_list[idx], reference])
```


2. Các model đang được sử dụng chính:

a) Retriever:
- Mặc định: "intfloat/e5-base-v2"
- Có thể được cấu hình trong file config:


```3:7:project-flashrag/flashrag/config/basic_config.yaml
model2path:
  e5: "intfloat/e5-base-v2"
  bge: "intfloat/e5-base-v2"
  contriever: "facebook/contriever"
  llama2-7B-chat: "meta-llama/Llama-2-7b-chat-hf"
```


b) Reranker (tùy chọn):
- Có thể được cấu hình trong file config:


```63:70:project-flashrag/examples/methods/my_config.yaml
use_reranker: False # whether to use reranker
rerank_model_name: ~ # same as retrieval_method
rerank_model_path: ~ # path to reranker model, path will be automatically find in `retriever_model2path`
rerank_pooling_method: ~
rerank_topk: 5  # number of remain documents after reranking
rerank_max_length: 512
rerank_batch_size: 256 # batch size for reranker
rerank_use_fp16: True
```


c) Generator (LLM):
- Mặc định: "PartAI/Dorna-Llama3-8B-Instruct"
- Có thể được cấu hình trong file config:


```73:80:project-flashrag/examples/methods/my_config.yaml
framework: vllm # inference frame work of LLM, supporting: 'hf','vllm','fschat'
generator_model: "llama3-8B-instruct" # name or path of the generator model
generator_max_input_len: 2048  # max length of the input
generator_batch_size: 2 # batch size for generation, invalid for vllm
generation_params:
  do_sample: False
  max_tokens: 32
use_fid: False # whether to use FID, only valid in encoder-decoder model
```


Ngoài ra, project cũng hỗ trợ sử dụng các mô hình GPT thông qua API OpenAI hoặc Azure, như được thể hiện trong file:


```77:154:project-flashrag/deepeval/models/gpt_model.py
class GPTModel(DeepEvalBaseLLM):
    ...
    def load_model(self):
        if self.should_use_azure_openai():
            openai_api_key = KEY_FILE_HANDLER.fetch_data(
                KeyValues.AZURE_OPENAI_API_KEY
            )
            openai_api_version = KEY_FILE_HANDLER.fetch_data(
                KeyValues.OPENAI_API_VERSION
            )
            azure_deployment = KEY_FILE_HANDLER.fetch_data(
                KeyValues.AZURE_DEPLOYMENT_NAME
            )
            azure_endpoint = KEY_FILE_HANDLER.fetch_data(
                KeyValues.AZURE_OPENAI_ENDPOINT
            )
            model_version = KEY_FILE_HANDLER.fetch_data(
                KeyValues.AZURE_MODEL_VERSION
            )

            if model_version is None:
                model_version = ""

            return AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=azure_deployment,
                azure_endpoint=azure_endpoint,
                openai_api_key=openai_api_key,
                model_version=model_version,
                *self.args,
                **self.kwargs,
            )
        elif self.should_use_local_model():
            model_name = KEY_FILE_HANDLER.fetch_data(KeyValues.LOCAL_MODEL_NAME)
            openai_api_key = KEY_FILE_HANDLER.fetch_data(
                KeyValues.LOCAL_MODEL_API_KEY
            )
            base_url = KEY_FILE_HANDLER.fetch_data(
                KeyValues.LOCAL_MODEL_BASE_URL
            )
            format = KEY_FILE_HANDLER.fetch_data(KeyValues.LOCAL_MODEL_FORMAT)
            return CustomChatOpenAI(
                model_name=model_name,
                openai_api_key=openai_api_key,
                base_url=base_url,
                # format=json to enable Ollama JSON mode
                format=format,
                # Hardcoded temperature to 0 to improve output JSON generation reliability
                temperature=0,
                *self.args,
                **self.kwargs,
            )
```


Tóm lại, project này sử dụng một pipeline RAG linh hoạt với khả năng tùy chỉnh các mô hình retriever, reranker (tùy chọn) và generator thông qua cấu hình. Các mô hình chính được sử dụng là E5 cho retriever và Llama 3 cho generator, nhưng có thể được thay đổi dễ dàng thông qua cấu hình.

### Some Notes

1. Trong path file này: `flashrag/retriever/retriever.py`

```python
local_files_only = kwargs.get("local_files_only", False)
```

    Dòng này đang set giá trị mặc định của `local_files_only` là `False`. Điều này có nghĩa:

    1. Nếu không có tham số `local_files_only` được truyền vào qua `kwargs`, giá trị mặc định sẽ là `False`.

    2. Khi `local_files_only` là `False`, nó cho phép hệ thống tự động tải model từ internet nếu không tìm thấy model trong thư mục cache local.

    3. Điều này rất hữu ích khi bạn muốn hệ thống tự động tải model nếu chưa có sẵn, giúp quá trình setup dễ dàng hơn.

    4. Ngược lại, nếu bạn set `local_files_only=True` khi gọi hàm, nó sẽ chỉ sử dụng các file có sẵn trong máy và sẽ báo lỗi nếu không tìm thấy model locally.

Vì vậy, với giá trị mặc định là `False`, hệ thống sẽ tự động download model nếu cần thiết, đúng như bạn đã hiểu.